In [28]:
import pandas as pd
import numpy as np
import glob, os
import xlrd

In [4]:
#os.chdir("/home/sara/Documents/Immigration/Shared_statistics/")
resident_foreigners_norm = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/resident_foreigners_norm.csv")

province_capitals = sorted(list(set(resident_foreigners_norm["Province"])))

# Initialize an empty DataFrame for the distances-matrix
distances = pd.DataFrame(index = province_capitals, columns=province_capitals)

In [5]:
prov_codes = pd.read_table("/home/sara/Documents/Immigration/Shared_models/Data_distances/Elenco-comuni-italiani.csv", sep= ";", 
                           usecols = ['Codice Regione', 'Codice Città Metropolitana', 
                                      'Codice Comune formato numerico', 'Denominazione corrente'])

prov_codes['Denominazione corrente'].replace(['Verbania', 'Monza', 'Forlì', 'Massa', 'Pesaro', 'Aosta', 'Bolzano', 'Sanluri', 'Tempio Pausania', 'Barletta', 'Carbonia', 'Tortolì'], ['Verbano-Cusio-Ossola', 'Monza e della Brianza', 'Forlì-Cesena', 'Massa-Carrara', 'Pesaro e Urbino', "Valle d'Aosta / Vallée d'Aoste", 'Bolzano / Bozen', 'Medio Campidano', 'Olbia-Tempio', 'Barletta-Andria-Trani', 'Carbonia-Iglesias', 'Ogliastra'], inplace = True)
#prov_codes['Denominazione provincia'].replace(['Bolzano/Bozen', "Valle d'Aosta/Vallée d'Aoste"], ['Bolzano / Bozen', "Valle d'Aosta / Vallée d'Aoste"], inplace = True)

In [6]:
prov_codes.head()

,Codice Regione,Codice Città Metropolitana,Denominazione corrente,Codice Comune formato numerico
0,1.0,201,Agliè,1001.0
1,1.0,201,Airasca,1002.0
2,1.0,201,Ala di Stura,1003.0
3,1.0,201,Albiano d'Ivrea,1004.0
4,1.0,201,Alice Superiore,1005.0


In [7]:
prov_codes = prov_codes[prov_codes["Denominazione corrente"].isin(province_capitals)]
prov_codes.index = range(len(prov_codes))

In [8]:
prov_codes.head()

,Codice Regione,Codice Città Metropolitana,Denominazione corrente,Codice Comune formato numerico
0,1.0,201,Torino,1272.0
1,1.0,-,Vercelli,2158.0
2,1.0,-,Novara,3106.0
3,1.0,-,Cuneo,4078.0
4,1.0,-,Asti,5005.0


In [9]:
all_codes = prov_codes["Codice Comune formato numerico"].values

In [10]:
name_codes_corr = prov_codes.set_index('Codice Comune formato numerico')['Denominazione corrente'].to_dict()

In [14]:
all_distances = {}
os.chdir("/home/sara/Documents/Immigration/Shared_models/Data_distances/Regions")
for file in glob.glob("*.txt"):
    print(file)

Abruzzo.txt
Basilicata.txt
Calabria.txt
Campania.txt
Emilia.txt
Friuli.txt
Lazio.txt
Liguria.txt
Lombardia.txt
Marche.txt
Molise.txt
Piemonte.txt
Puglia.txt
Sardegna.txt
Sicilia.txt
Toscana.txt
Trentino.txt
Umbria.txt
ValledAosta.txt
Veneto.txt


In [15]:
for file in glob.glob("*.xls"):
    print(file)

Dist_porti.xls


In [16]:
a = pd.read_table("Abruzzo.txt", sep = ";", usecols = ["origine", "Destinazione", "Total_Mete"])
a.rename(columns={'origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)

for c in a.columns:
    a[c] = a[c].str.replace(",", ".")
    a[c] = a[c].astype(float)
    
a = a[(a["Origin"].isin(all_codes)) & (a["Destination"].isin(all_codes))]
a = a.replace(name_codes_corr)

a = a.pivot(index='Origin', columns='Destination', values='Distance')

In [19]:
try:
    b = pd.read_table("Basilicata.txt", sep = ";", usecols = ["Origine", "Destinazione", "Total_Mete"])
    b.rename(columns={'Origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)
except ValueError:
    b = pd.read_table("Basilicata.txt", sep = ";", usecols = ["origine", "Destinazione", "Total_Mete"])
    b.rename(columns={'origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)
    


for c in b.columns:
    b[c] = b[c].str.replace(",", ".")
    b[c] = b[c].astype(float)
    
b = b[(b["Origin"].isin(all_codes)) & (b["Destination"].isin(all_codes))]
b = b.replace(name_codes_corr)

b = b.pivot(index='Origin', columns='Destination', values='Distance')

In [23]:
c = pd.concat([a, b], axis=1)

In [32]:
aa = pd.read_excel("Dist_porti.xls")

In [33]:
aa

,Collegamento per le isole minori o ai comuni non connessi,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Name,Origine,Den_orig,Destinazione,Den_dest,Total_Minu,Total_Mete
1,49012 - 49014,49012,Piombino,49014,Portoferraio,97,29800
2,49009 - 49005,49009,Livorno,49005,Isola di Capraia,226,75700
3,53016 - 53012,53016,Monte Argentario,53012,Isola del Giglio,120,42000
4,63049 - 63037,63049,Napoli,63037,Ischia Porto,118,49000
5,63049 - 63014,63049,Napoli,63014,Capri,109,53600
6,63049 - 63061,63049,Napoli,63061,Procida,80,31400
7,58032 - 104017,58032,Civitavecchia,104017,Olbia,360,237000
8,49009 - 104017,49009,Livorno,104017,Olbia,540,315000
9,58007 - 59018,58007,Anzio,59018,Ponza,194,75000


In [35]:
a = pd.read_table("Sardegna.txt", sep = ";", usecols = ["Origine", "Destinazione", "Total_Mete"])
a.rename(columns={'Origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)

for c in a.columns:
    a[c] = a[c].str.replace(",", ".")
    a[c] = a[c].astype(float)
    
a = a[(a["Origin"].isin(all_codes)) & (a["Destination"].isin(all_codes))]
a = a.replace(name_codes_corr)

a = a.pivot(index='Origin', columns='Destination', values='Distance')

In [36]:
a

Destination,Cagliari,Nuoro,Oristano,Sassari
Origin,,,,
Cagliari,0.00,178918.64,92792.05,212663.61
Nuoro,180215.42,0.00,89769.42,120022.83
Oristano,92113.04,88320.92,0.00,122065.89
Sassari,211277.04,120133.83,120831.04,0.00
